<a href="https://colab.research.google.com/github/Paulo-vitorCS/DeepLearning/blob/main/Projeto_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projeto 05: Classificação binária breast cancer - carregar classificador

###Etapa 01: Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

###Etapa 02: Carregamento do classificador

In [2]:
np.random.seed(123)
torch.manual_seed(123)

In [3]:
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(30, 8)
        # Mesmos parâmetros utilizados pelo kernel initializer do Keras
        torch.nn.init.normal_(self.dense0.weight, mean = 0.0, std = 0.05)
        self.dense1 = nn.Linear(8, 8)
        torch.nn.init.normal_(self.dense1.weight, mean = 0.0, std = 0.05)
        self.dense2 = nn.Linear(8, 1)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.output = nn.Sigmoid()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense2(X)
        X = self.output(X)
        return X

In [4]:
classificador = classificador_torch()

In [7]:
state_dict = torch.load('/content/checkpoint.pth')  # Carregando o classificador

In [8]:
state_dict

OrderedDict([('dense0.weight',
              tensor([[-6.3329e-01, -6.4088e-03, -3.8681e-01, -1.0687e-01,  9.4179e-02,
                       -2.8071e-02,  5.8217e-02, -5.5828e-02,  1.4919e-02,  5.5338e-02,
                        8.5171e-03, -1.0727e-03,  3.1089e-03, -1.7085e-02,  3.1640e-02,
                        2.3500e-01, -3.9340e-01, -1.7852e-01, -6.7033e-02,  5.4838e-01,
                       -4.7472e-01,  2.4388e-02, -3.2421e-01,  1.8188e-01,  1.4345e-02,
                        3.0411e-02,  1.2222e-02,  4.2423e-02,  1.6629e-02,  6.2451e-01],
                      [ 9.6147e-02,  8.9823e-03,  6.2657e-02, -2.7833e-02,  2.5520e-02,
                       -1.4683e-02,  8.4852e-09, -5.5642e-02, -4.3206e-02,  2.9459e-04,
                        3.3087e-02,  1.5457e-03, -3.6564e-02,  7.3219e-02, -9.4868e-11,
                       -5.2539e-07,  3.7381e-03,  2.0600e-09,  6.6002e-09,  2.5737e-10,
                       -7.0341e-02,  3.3297e-02, -3.6295e-02, -3.3981e-02, -7.6689e-02,


In [9]:
classificador.load_state_dict(state_dict)  # Carregar tudo que está na variável state_dict

<All keys matched successfully>

###Etapa 3: Previsões

In [10]:
new = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]])

In [11]:
classificador.eval()  # modo de avaliação
previsao = classificador(new)
previsao = (previsao.detach().numpy() > 0.5)  # Tratamento para tirar informação adicional da previsão
previsao

array([[ True]])

In [12]:
# Previsões a partir de uma base de dados
previsores = pd.read_csv('/content/entradas_breast.csv')
classe = pd.read_csv('/content/saidas_breast.csv')

In [13]:
# Conversão
previsores = torch.tensor(np.array(previsores), dtype = torch.float)
classe = torch.tensor(np.array(classe), dtype = torch.float)

In [14]:
previsoes = classificador.forward(previsores).detach()

In [15]:
previsoes

tensor([[5.2292e-07],
        [9.2073e-05],
        [1.0848e-03],
        [1.5193e-04],
        [1.2827e-02],
        [5.7314e-01],
        [7.7519e-05],
        [2.3465e-01],
        [3.0953e-01],
        [6.9902e-03],
        [1.2647e-01],
        [4.5154e-03],
        [2.5287e-01],
        [6.8647e-01],
        [6.8647e-01],
        [4.6997e-01],
        [3.0779e-02],
        [3.7884e-03],
        [3.9827e-08],
        [9.6103e-01],
        [6.8647e-01],
        [1.0000e+00],
        [3.9961e-01],
        [2.4730e-08],
        [1.7898e-08],
        [2.7582e-04],
        [4.2254e-01],
        [1.6693e-02],
        [1.7407e-03],
        [1.9918e-01],
        [1.4824e-03],
        [5.1415e-02],
        [1.2670e-02],
        [7.5736e-05],
        [5.3576e-03],
        [9.8634e-02],
        [6.8647e-01],
        [9.9928e-01],
        [6.8647e-01],
        [6.3165e-01],
        [6.6119e-01],
        [8.2413e-01],
        [1.3902e-04],
        [1.4843e-01],
        [6.8647e-01],
        [1

In [16]:
F.binary_cross_entropy(previsoes, classe).numpy()  # Retorno do valor do erro na base de dados 

array(0.1990281, dtype=float32)

In [17]:
accuracy_score(classe.numpy(), (previsoes > 0.5).numpy())

0.9332161687170475